# Modeling Pipeline and AutoML
## Load data and packages

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import mlflow
import mlflow.sklearn

import h2o
from h2o.automl import H2OAutoML

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix

In [3]:
train_data = pd.read_csv('data/processed/train_data.csv')
test_data  = pd.read_csv('data/processed/test_data.csv')

In [4]:
train_data = train_data.drop(columns = ['Unnamed: 0','product_name'])
test_data = test_data.drop(columns = ['Unnamed: 0','product_name'])

In [5]:
X_train = train_data.drop(columns=['reordered','product_id','user_id'])
y_train = train_data['reordered']

X_test  = test_data.drop(columns=['reordered','product_id','user_id'])
y_test  = test_data['reordered']

## Preprocessing

In [7]:
categorical_cols = ['aisle_id', 'department_id']
numeric_cols = [col for col in X_train.columns if col not in categorical_cols]

In [8]:
# Transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

## Modeling

Since our model is trying to predict `reordered`, which is a binary variable, we will choose F1 score as our main metrics, which balances between precision and recall. We will also look at accuracy as well.

### Baseline Model (Random Forest)

In [10]:
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=32021, class_weight='balanced'))
])

In [11]:
# Log baseline results with MLFlow
with mlflow.start_run(run_name='Baseline_RandomForest_Balanced'):
    rf_pipeline.fit(X_train, y_train)
    y_pred = rf_pipeline.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    mlflow.log_param('Model', 'RandomForest Balanced Data')
    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy)
    mlflow.sklearn.log_model(rf_pipeline, 'baseline_rf_pipeline_balanced')

    # Confusion matrix plot
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title('Confusion Matrix for Baseline RandomForest with Balanced Data')
    cm_path = 'confusion_matrix_rf_balanced.png'
    plt.savefig(cm_path)
    plt.close(fig)
    mlflow.log_artifact(cm_path)
    
print('Baseline RF logged.')

2025/08/20 18:43:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/20 18:43:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Baseline RF logged.


### H2O AutoML

In [13]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "13" 2019-09-17; Java(TM) SE Runtime Environment (build 13+33); Java HotSpot(TM) 64-Bit Server VM (build 13+33, mixed mode, sharing)
  Starting server from /opt/anaconda3/envs/MLOps/lib/python3.13/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kf/8ms7_h8j11bf86d6c5ykldrw0000gn/T/tmpcmazrn_h
  JVM stdout: /var/folders/kf/8ms7_h8j11bf86d6c5ykldrw0000gn/T/tmpcmazrn_h/h2o_quynhanhnd2402_started_from_python.out
  JVM stderr: /var/folders/kf/8ms7_h8j11bf86d6c5ykldrw0000gn/T/tmpcmazrn_h/h2o_quynhanhnd2402_started_from_python.err
  Server is running at http://127.0.0.1:54321
 successful.o H2O server at http://127.0.0.1:54321 ...
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,4 months and 24 days
H2O_cluster_name:,H2O_from_python_quynhanhnd2402_xtprzf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [14]:
train_h2o = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
test_h2o  = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))

train_h2o['reordered'] = train_h2o['reordered'].asfactor()
test_h2o['reordered']  = test_h2o['reordered'].asfactor()

train_h2o['aisle_id'] = train_h2o['aisle_id'].asfactor()
train_h2o['department_id'] = train_h2o['department_id'].asfactor()

test_h2o['aisle_id'] = test_h2o['aisle_id'].asfactor()
test_h2o['department_id'] = test_h2o['department_id'].asfactor()

train, valid = train_h2o.split_frame(ratios=[0.8], seed=32021)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [15]:
target = 'reordered'
features = numeric_cols + categorical_cols

In [16]:
aml = H2OAutoML(max_models=20, seed=32021, exclude_algos = ['StackedEnsemble', 'DeepLearning'], nfolds=0)
aml.train(x=features, y=target, training_frame=train, validation_frame=valid)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_3_AutoML_1_20250820_184414


Model Summary: 
    number_of_trees
--  -----------------
    950

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.10401797121165507
RMSE: 0.32251817190920434
LogLoss: 0.3277231648008529
Mean Per-Class Error: 0.19405817068891604
AUC: 0.9507999332057575
AUCPR: 0.5803704901008337
Gini: 0.901599866411515

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7448951904827723
       0       1      Error    Rate
-----  ------  -----  -------  ------------------
0      363681  17800  0.0467   (17800.0/381481.0)
1      9094    17539  0.3415   (9094.0/26633.0)
Total  372775  35339  0.0659   (26894.0/408114.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.744895     0.56603   76
max f2                       0.616007     0.677573  120
max f0point5                 0.857287     0.571939  38
max accuracy                 0.867541     0.946429  34
max precision                0.9914       1         0
max recall                   0.0831171    1         337
max specificity              0.9914       1         0
max absolute_mcc             0.710655     0.53952   87
max min_per_class_accuracy   0.569591     0.882217  136
max mean_per_class_accuracy  0.506891     0.887145  158
max tns                      0.9914       381481    0
max fns                      0.9914       26535     0
max fps                      0.000297075  381481    399
max tps                      0.0831171    26633     337
max tnr                      0.9914       1         0
max fnr                      0.9914       0.99632   0
max fpr                      0.000297075  1         399
max tpr                      0.0831171    1         337

Gains/Lift Table: Avg response rate:  6.53 %, avg score: 25.99 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100021                   0.937702           12.403      12.403             0.809407         0.958344   0.809407                    0.958344            0.124057        0.124057                   1140.3    1140.3             0.122017
2        0.0200018                   0.908173           10.1644     11.2839            0.663318         0.922726   0.736371                    0.940537            0.101641        0.225697                   916.443   1028.39            0.220056
3        0.0300014                   0.880935           9.15813     10.5754            0.597648         0.894464   0.690134                    0.925181            0.0915781       0.317276                   815.813   957.535            0.30733
4        0.0400011                   0.854748           7.74254     9.86719            0.505268         0.867494   0.64392                     0.91076             0.0774227       0.394698                   674.254   886.719            0.37946
5        0.0500007                   0.830157           6.66114     9.22601            0.434697         0.842342   0.602078                    0.897077            0.0666091       0.461307                   566.114   822.601            0.440022
6        0.100001                    0.712574           5.02302     7.12452            0.327796         0.770723   0.464937                    0.8339              0.251155        0.712462                   402.302   612.452            0.655219
7        0.15                        0.604308          

#### Insights and best model

In [18]:
lb = aml.leaderboard
print(lb.head(10))

best_model = aml.leader
print(best_model)

model_id                                              auc    logloss     aucpr    mean_per_class_error      rmse        mse
XGBoost_3_AutoML_1_20250820_184414               0.840823   0.376077  0.340135                0.300338  0.348989  0.121794
XGBoost_grid_1_AutoML_1_20250820_184414_model_2  0.839789   0.185635  0.345569                0.293925  0.225061  0.0506526
XGBoost_grid_1_AutoML_1_20250820_184414_model_5  0.837796   0.186441  0.341477                0.301487  0.225301  0.0507605
XGBoost_grid_1_AutoML_1_20250820_184414_model_6  0.831625   0.189317  0.321966                0.311297  0.227037  0.0515459
XGBoost_grid_1_AutoML_1_20250820_184414_model_4  0.823079   0.193015  0.299415                0.319264  0.229271  0.0525651
XGBoost_grid_1_AutoML_1_20250820_184414_model_3  0.822749   0.261164  0.299321                0.322373  0.274199  0.0751849
GBM_grid_1_AutoML_1_20250820_184414_model_3      0.821725   0.194081  0.294721                0.325819  0.230053  0.0529246
XGBoost_2

#### Predict on test set and log results

In [20]:
preds = best_model.predict(test_h2o)
pred_labels = preds.as_data_frame()['predict']

f1 = f1_score(y_test, pred_labels)
accuracy = accuracy_score(y_test, pred_labels)

cm = confusion_matrix(y_test, pred_labels)
fig, ax = plt.subplots(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title("Confusion Matrix for H2O AutoML's best model with Balanced data")
cm_path = 'confusion_matrix_aml_balanced.png'
plt.savefig(cm_path)
plt.close(fig)

# Log automl's best model results with mlflow
with mlflow.start_run(run_name='H2O_AutoML_Balanced'):
    model_path = h2o.save_model(best_model, path='models', force=True)
    mlflow.log_artifact(model_path, 'h2o_aml_model_balanced')

    mlflow.log_param('model_type', 'H2O_AutoML_Leader_Balanced')
    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy)

    mlflow.log_artifact(cm_path)

print('H2O AutoML results logged.')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


/opt/anaconda3/envs/MLOps/lib/python3.13/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


H2O AutoML results logged.
